In [ ]:
import pandas as pd
# import pandasql as ps
import matplotlib.pyplot as plt
# Import argsort
from numpy import argsort
from Functions.connection.connection import *
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.xgboost import *
from Functions.Models.evaluation import *
from Functions.NLP.alertp1_nlp import *
from Functions.NLP.data_with_nlp import *
from Functions.pipeline import *

In [ ]:
alertP1=connection("credentials.txt")

In [ ]:
data = pre_process(alertP1)

In [ ]:
def ner_features(df, column):
    import spacy
    from sklearn.preprocessing import MultiLabelBinarizer
    nlp = spacy.load('pt_core_news_md')
    ner = []
    for doc in nlp.pipe(df[column]):
        doc_ner = []
        for ent in doc.ents:
            doc_ner.append((ent.text, ent.label_))
        ner.append(doc_ner)
    df['ner'] = ner
    return df

In [ ]:
new_data = ner_features(data, "clean_text")

In [ ]:
new_data

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

def one_hot_encode_ner(df):
    mlb = MultiLabelBinarizer()
    ner_one_hot = mlb.fit_transform(df['ner'].apply(lambda x: [label for _, label in x]))
    ner_df = pd.DataFrame(ner_one_hot, columns=mlb.classes_)
    return pd.concat([df, ner_df], axis=1)

In [ ]:
new_data = new_data.reset_index(drop=True)
new_data = one_hot_encode_ner(new_data)
new_data = new_data.drop('ner', axis=1)

In [ ]:
eliminate_cols=['ID_DOENTE','PROCESSO','COD_REFERENCIA','result','COD_PZ','COD_UNID_SAUDE_PROV','UNID_PROV','TIPO_UNID','COD_CTH_PRIOR','CTH_PRIOR','COD_MOTIVO_RECUSA','DES_MOTIVO_RECUSA','COD_ESPECIALIDADE','DES_ESPECIALIDADE','agrupadora','OUTRA_ENTIDADE','DATA_RECEPCAO','DATA_ENVIO','DATA_RETORNO','NUM_TAXA','ESTADO','DATA_MARCACAO','DATA_REALIZACAO','OBSERVACOES','Mês_entrada','Ano_entrada','trata data recusa','resume saída','mês_saida','ano_saida','Texto','clean_text']
X = new_data.drop(eliminate_cols,axis=1)# Features
y = new_data.result # Target variable
features=X.columns

y_pred_train, y_pred_test, model_score, X_train, X_test, y_train, y_test = xgb_classifier(X,y)

In [ ]:
from xgboost import plot_importance

# Plot the feature importance scores
plot_importance(model_score)
plt.show()

In [ ]:
evaluate_thresholds(y_pred_train,y_train,y_pred_test,y_test,metrics="test")


### TRYING TOPIC LDA

In [ ]:
def get_top_words(lda, vectorizer, n_top_words=10):
    # Extract top words for each topic
    top_words = []
    for topic_idx, topic in enumerate(lda.components_):
        top_words.append([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-n_top_words - 1:-1]])

    # Print top words for each topic
    for i, words in enumerate(top_words):
        print(f'Topic {i}: ' + ', '.join(words))

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

def lda_topics(df, column, n_components=5):
    # Convert text data into a bag of words representation
    vectorizer = CountVectorizer(analyzer='word',min_df=10)
    data_vectorized = vectorizer.fit_transform(df[column])

    # Fit LDA model to the data
    lda = LatentDirichletAllocation(n_components=n_components)
    lda.fit(data_vectorized)

    # Transform data to get topic distribution for each row
    topic_dist = lda.transform(data_vectorized)

    # Create new dataframe with topic columns
    topic_columns = [f'topic_{i}' for i in range(n_components)]
    topic_df = pd.DataFrame(topic_dist, columns=topic_columns)

    # Concatenate topic dataframe with original dataframe
    df = pd.concat([df.reset_index(drop=True), topic_df], axis=1)
                   
    get_top_words(lda, vectorizer)

    return df

In [ ]:
lda_data = lda_topics(data,"clean_text")
lda_data = lda_data.drop('ner', axis=1)



In [ ]:
lda_data

In [ ]:
eliminate_cols=['ID_DOENTE','PROCESSO','COD_REFERENCIA','result','COD_PZ','COD_UNID_SAUDE_PROV','UNID_PROV','TIPO_UNID','COD_CTH_PRIOR','CTH_PRIOR','COD_MOTIVO_RECUSA','DES_MOTIVO_RECUSA','COD_ESPECIALIDADE','DES_ESPECIALIDADE','agrupadora','OUTRA_ENTIDADE','DATA_RECEPCAO','DATA_ENVIO','DATA_RETORNO','NUM_TAXA','ESTADO','DATA_MARCACAO','DATA_REALIZACAO','OBSERVACOES','Mês_entrada','Ano_entrada','trata data recusa','resume saída','mês_saida','ano_saida','Texto','clean_text']
X = lda_data.drop(eliminate_cols,axis=1)# Features
y = lda_data.result # Target variable
features=X.columns

y_pred_train, y_pred_test, model_score, X_train, X_test, y_train, y_test = xgb_classifier(X,y)

In [ ]:
from xgboost import plot_importance

# Plot the feature importance scores
plot_importance(model_score)
plt.show()
evaluate_thresholds(y_pred_train,y_train,y_pred_test,y_test,metrics="test")

In [ ]:
lda_data = lda_topics(data,"clean_text",20)
lda_data = lda_data.drop('ner', axis=1)
eliminate_cols=['ID_DOENTE','PROCESSO','COD_REFERENCIA','result','COD_PZ','COD_UNID_SAUDE_PROV','UNID_PROV','TIPO_UNID','COD_CTH_PRIOR','CTH_PRIOR','COD_MOTIVO_RECUSA','DES_MOTIVO_RECUSA','COD_ESPECIALIDADE','DES_ESPECIALIDADE','agrupadora','OUTRA_ENTIDADE','DATA_RECEPCAO','DATA_ENVIO','DATA_RETORNO','NUM_TAXA','ESTADO','DATA_MARCACAO','DATA_REALIZACAO','OBSERVACOES','Mês_entrada','Ano_entrada','trata data recusa','resume saída','mês_saida','ano_saida','Texto','clean_text']
X = lda_data.drop(eliminate_cols,axis=1)# Features
y = lda_data.result # Target variable
features=X.columns

y_pred_train, y_pred_test, model_score, X_train, X_test, y_train, y_test = xgb_classifier(X,y)

In [ ]:
from xgboost import plot_importance

# Plot the feature importance scores
plot_importance(model_score)
plt.show()
evaluate_thresholds(y_pred_train,y_train,y_pred_test,y_test,metrics="test")


In [ ]:
lda_data = lda_topics(data,"clean_text",100)
lda_data = lda_data.drop('ner', axis=1)
eliminate_cols=['ID_DOENTE','PROCESSO','COD_REFERENCIA','result','COD_PZ','COD_UNID_SAUDE_PROV','UNID_PROV','TIPO_UNID','COD_CTH_PRIOR','CTH_PRIOR','COD_MOTIVO_RECUSA','DES_MOTIVO_RECUSA','COD_ESPECIALIDADE','DES_ESPECIALIDADE','agrupadora','OUTRA_ENTIDADE','DATA_RECEPCAO','DATA_ENVIO','DATA_RETORNO','NUM_TAXA','ESTADO','DATA_MARCACAO','DATA_REALIZACAO','OBSERVACOES','Mês_entrada','Ano_entrada','trata data recusa','resume saída','mês_saida','ano_saida','Texto','clean_text']
X = lda_data.drop(eliminate_cols,axis=1)# Features
y = lda_data.result # Target variable
features=X.columns

y_pred_train, y_pred_test, model_score, X_train, X_test, y_train, y_test = xgb_classifier(X,y)

In [ ]:
from xgboost import plot_importance

# Plot the feature importance scores
plot_importance(model_score)
plt.show()
evaluate_thresholds(y_pred_train,y_train,y_pred_test,y_test,metrics="test")